# Make a simple FeedForward network for the bus data

## Initialization

In [1]:
import sys
sys.path.append('../misc')

from MoviaBusDataset import MoviaBusDataset
from BaseNetwork import BaseNetwork
import torch
from torch.nn import Linear, Sequential, ReLU, L1Loss
from torch.optim import Adam, lr_scheduler
import numpy as np

## Import data using the MoviaBusDataset

In [2]:
previous_timesteps = 6
prediction_steps = 3
batch_size = 25
num_epochs = 100

train = MoviaBusDataset('../data/train', interpolation=True, prev_timesteps=previous_timesteps, max_future_time_steps=prediction_steps, timeofday = True)
test = MoviaBusDataset('../data/test', interpolation=True, prev_timesteps=previous_timesteps, max_future_time_steps=prediction_steps, timeofday = True)

train.normalize()
test.normalize(train.mean, train.std)

## The Model

In [3]:
class FNN(BaseNetwork):
    def __init__(self, num_hidden):
        super().__init__()
        

        self.FNN = Sequential(
            Linear(previous_timesteps+1, num_hidden),
            ReLU(),
            Linear(num_hidden, num_hidden),
            ReLU(),
            Linear(num_hidden, 1),
        )

    def forward(self,x):
        """
        x : [batch_size, prev_timesteps, num_roads]
        
        """
        #Transpose input, such that the previous time steps are the last dimension
        x = x.transpose(2,1)

        predictions = []
        for _ in range(self.max_timestep):
            #Run the input through the network
            prediction = self.FNN(x).squeeze()

            #Append the prediction to the list of predictions. 
            #If the data includes timeofday, this shouldn't be included
            predictions.append(prediction[:,:self.num_roads])

            #remove oldest timestep
            x = x[:,:,1:]
            #unsqueeze output so its size is [batch_size, num_roads, timesteps]
            prediction = prediction.unsqueeze(2)

            #append the new prediction to the input
            x = torch.cat((x,prediction),dim=2)


        return torch.stack(predictions,1)

net = FNN(num_hidden=100)

optimizer_fun = lambda param : Adam(param, lr=1e-2, weight_decay=1e-4)
scheduler_fun = lambda optim : lr_scheduler.StepLR(optim, step_size=10, gamma=0.1)
criterion = L1Loss()

In [4]:
train[2]

{'data': tensor([[ 0.5934, -0.1722,  0.3096,  ...,  0.5887, -0.3316,  0.2803],
         [-1.5129, -0.4114,  1.5964,  ...,  0.4903,  1.6398,  0.2841],
         [-0.3628,  0.8573, -0.7509,  ...,  0.2151,  0.7666,  0.2879],
         ...,
         [-1.0357,  0.8193,  0.4174,  ..., -1.0039,  1.7890,  0.2955],
         [ 0.5031,  1.1165, -0.3916,  ..., -0.6121,  0.1225,  0.2992],
         [ 0.5257,  0.9196, -0.2602,  ..., -0.2203, -1.5440,  0.3030]]),
 'target': tensor([[ 0.7827,  0.5483, -0.6762, -0.3714, -0.4209, -0.2062,  0.6613,  0.0310,
          -0.3420, -4.4938,  0.3545, -1.3696,  0.2946, -0.9763, -0.3234,  0.5122,
           0.2355,  0.0938, -0.3928, -0.1647,  0.5333, -1.8450, -0.2074, -1.2911,
          -0.0143, -0.7088,  0.2245,  0.6666,  0.4820,  0.5480,  0.3637,  0.4563,
           2.1519,  2.3912, -0.6493,  2.1873,  2.4583,  0.7182,  1.9328,  0.6199,
          -1.1852, -0.1627, -0.7424,  1.0879, -1.5996, -0.2027, -0.2765, -0.2142,
           0.8565, -0.4696, -0.3290, -0.4134, -0

## Training the network

In [5]:
net.train_network(train, test,batch_size=batch_size, 
                  num_epochs=30,
                  optimizer_fun=optimizer_fun,
                  scheduler_fun=scheduler_fun,
                  criterion=criterion)

epoch =  0  train loss = 0.733   test loss = 0.718   output_std = 0.296
epoch =  2  train loss = 0.698   test loss = 0.708   output_std = 0.332
epoch =  4  train loss = 0.694   test loss = 0.704   output_std = 0.355
epoch =  6  train loss = 0.691   test loss = 0.702   output_std = 0.366
epoch =  8  train loss = 0.690   test loss = 0.700   output_std = 0.370
epoch = 10  train loss = 0.689   test loss = 0.700   output_std = 0.377
epoch = 12  train loss = 0.681   test loss = 0.686   output_std = 0.412
epoch = 14  train loss = 0.681   test loss = 0.685   output_std = 0.421
epoch = 16  train loss = 0.680   test loss = 0.685   output_std = 0.424
epoch = 18  train loss = 0.680   test loss = 0.684   output_std = 0.427
epoch = 20  train loss = 0.680   test loss = 0.684   output_std = 0.427
epoch = 22  train loss = 0.679   test loss = 0.684   output_std = 0.427
epoch = 24  train loss = 0.679   test loss = 0.684   output_std = 0.428
epoch = 26  train loss = 0.679   test loss = 0.684   output_std 

## Evaluate the network on a test dataset 

In [6]:
net.get_MAE_score(timestep=1)

1.340066909790039

### Example of modelled data vs real data for one road segment 

In [7]:
%matplotlib notebook

net.visualize_road(timesteps=1, road=23)

OSError: [Errno 22] Invalid argument

In [16]:
train.dataframes[0].index[0:10].values.TotalSeconds()

AttributeError: 'numpy.ndarray' object has no attribute 'TotalSeconds'

In [25]:
import datetime
train.dataframes[0].index[0:10].values.astype(datetime.datetime)

array([1538373600000000000, 1538373900000000000, 1538374200000000000,
       1538374500000000000, 1538374800000000000, 1538375100000000000,
       1538375400000000000, 1538375700000000000, 1538376000000000000,
       1538376300000000000], dtype=object)

In [19]:
train.dataframes[0].index[0:10].values

array(['2018-10-01T06:00:00.000000000', '2018-10-01T06:05:00.000000000',
       '2018-10-01T06:10:00.000000000', '2018-10-01T06:15:00.000000000',
       '2018-10-01T06:20:00.000000000', '2018-10-01T06:25:00.000000000',
       '2018-10-01T06:30:00.000000000', '2018-10-01T06:35:00.000000000',
       '2018-10-01T06:40:00.000000000', '2018-10-01T06:45:00.000000000'],
      dtype='datetime64[ns]')